In [1]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 13.8MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621010 sha256=71666c15e4f65f5a645e03e5606e292a4656f514f71ffabb1e5a81bcdb3a0ccd
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=d177570112a8855f325b5adcedb0fc54d174d61c08c0777d93be12d3962cc1a0
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


In [3]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy
import time
import pycuda.driver as driver

NUM_PARTICLES = 10000
BLOCK_SIZE = 256
GRID_SIZE = (NUM_PARTICLES // BLOCK_SIZE if NUM_PARTICLES % BLOCK_SIZE == 0 else NUM_PARTICLES // BLOCK_SIZE + 1)
NUM_ITERATIONS = 10000

mod = SourceModule("""
  #include <curand_kernel.h>
  #include <curand.h>

  struct Particle {
    float position_x;
    float position_y;
    float position_z;

    float velocity_x;
    float velocity_y;
    float velocity_z;
  };

  extern "C" {
    __global__ void simulate(Particle* particles, int iterations) {
      int id = threadIdx.x + blockIdx.x * blockDim.x;
      Particle* p = &particles[id];

      curandState state;
      curand_init(id, id, 0, &state);

      for (int i = 0; i < iterations; i++) {
        p->velocity_x += curand_uniform(&state);
        p->velocity_y += curand_uniform(&state);
        p->velocity_z += curand_uniform(&state);

        p->position_x += p->velocity_x;
        p->position_y += p->velocity_y;
        p->position_z += p->velocity_z;
      }
    }
  }
  """, no_extern_c=True)


class ParticleStruct:
    mem_size = 6 * numpy.float32(0).nbytes
    def __init__(self, position, velocity, pointer):
      self.pointer = pointer
      cuda.memcpy_htod(pointer, position)
      cuda.memcpy_htod(pointer + numpy.float32(0).nbytes * (3), velocity)

    def __str__(self):
      return ("p: " + str(cuda.from_device(self.pointer, (3), numpy.float32)) + ", " +
                "v: " + str(cuda.from_device(self.pointer + numpy.float32(0).nbytes * (3), (3), numpy.float32)))


def simulate_gpu():
  driver.start_profiler()
  particle_pointer = cuda.mem_alloc(NUM_PARTICLES * ParticleStruct.mem_size)

  particles = []
  for i in range(NUM_PARTICLES):
    p = ParticleStruct(numpy.random.randn(3).astype(numpy.float32),
                        numpy.random.randn(3).astype(numpy.float32),
                        int(particle_pointer) + i * ParticleStruct.mem_size)
    particles.append(p)

  print("GPU initial:", list(map(lambda p: str(p), particles)))
  func = mod.get_function("simulate")
  func(particle_pointer, numpy.int32(NUM_ITERATIONS), grid=(GRID_SIZE,1), block=(BLOCK_SIZE,1,1))
  print("GPU result:", list(map(lambda p: str(p), particles)))
  driver.stop_profiler()


def simulate_cpu():
  velocity = numpy.random.randn(NUM_PARTICLES, 3)
  position = numpy.random.randn(NUM_PARTICLES, 3)

  print("CPU initial v:", velocity)
  print("CPU initial p:", position)

  for i in range(NUM_ITERATIONS):
    rand = numpy.random.randn(NUM_PARTICLES, 3)
    velocity += rand
    position += velocity

  print("CPU result v:", velocity)
  print("CPU result p:", position)


gpu_time_pre = time.time()
simulate_gpu()
gpu_execution_time = time.time() - gpu_time_pre

cpu_time_pre = time.time()
simulate_cpu()
cpu_execution_time = time.time() - gpu_time_pre

print("GPU execution time", gpu_execution_time)
print("CPU execution time", cpu_execution_time)


GPU initial: ['p: [-0.04833196 -1.0114126   1.5256994 ], v: [ 1.0342134  -1.8132185   0.20674452]', 'p: [0.40729022 0.30746996 0.35647932], v: [-0.18101686  1.0960544   0.23867819]', 'p: [1.88442   0.6008239 0.0791083], v: [ 1.1792562  -0.14425997  1.1326487 ]', 'p: [-1.2786664   0.49959415 -1.2942598 ], v: [-0.7301291  0.8621184 -1.1938976]', 'p: [ 0.2569345  -0.59913087 -1.2748423 ], v: [ 0.2576525  -1.7115952   0.37993008]', 'p: [-1.7667977 -0.0564154  1.1723384], v: [0.12138138 0.9894185  0.780387  ]', 'p: [-0.28731668 -2.3479838  -0.01722359], v: [0.23671342 0.41721255 0.03612928]', 'p: [-0.16729458 -0.11950558  1.80239   ], v: [ 0.842855   -1.0362139   0.17030358]', 'p: [-0.34855124 -1.1812526   1.0767431 ], v: [0.44607282 1.4776994  0.15397069]', 'p: [-1.0595564  -0.6260577  -0.39346594], v: [0.7415353  0.08667523 0.60770315]', 'p: [-0.8619594 -1.7260405  0.7990342], v: [-0.06037187  0.16956323 -1.2915661 ]', 'p: [ 0.6049106  -0.43428078 -0.65030015], v: [-1.0677304  1.0339984  